In [1]:
import pymongo
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import string
from slimit import ast
from slimit.parser import Parser
from slimit.visitors import nodevisitor
from slimit import minify

In [2]:
# connecting to my mongodb and get data out
client = pymongo.MongoClient('mongodb://localhost:27017/')
# create database
db = client.movie_db
# create colletions
col = db['rotten_tomato']

In [3]:
# get pending movie list fron mongo db
def get_latest_pending_list():
    tomato_data = pd.DataFrame(list(col.find()))
    goal_list = tomato_data.loc[tomato_data['critic(%)'] == 'na'][['name', 'url', 'year']].reset_index()
    
    return goal_list

In [4]:
pending_list = get_latest_pending_list()
pending_list

,index,name,url,year
0,6,The Avengers,https://www.rottentomatoes.com/m/the_avengers,2012
1,59,It,https://www.rottentomatoes.com/m/it,2017


In [5]:
#search movie in rotten tomato
base_url = 'https://www.rottentomatoes.com/search/?search='
# request url
def get_html(url):
    try:
        r = requests.get(url, timeout=120)
        r.raise_for_status
        #site encoding
        r.encoding = 'utf-8'
        
        return r.text
    except:
        print(url + 'ERROR')
        pass
    
def get_movie_list(url):
    html = get_html(url)
    soup = bs(html, 'lxml')
    movie_list = {}
    try:
        movie_list = soup.find_all('script')[32].text
    except:
        movie_list = 'na'
        
    return movie_list

def get_movie_url(data, movie, year):
    text = data
    parser = Parser()
    tree = parser.parse(text)
    # put the node data into a dict
    fields = []
    url_s = 'to find'
    for node in nodevisitor.visit(tree):
        if isinstance(node, ast.Assign):
            try:
                field = {str(node.left.value).replace('"',''): str(node.right.value).replace('"','')}
                fields.append(field)   
            except:
                pass
    # check title and year
    #print(fields)
    for index, field in enumerate(fields):
        try:
            if(field['name'] == movie):
                year_dic = fields[index + 1]
                url = fields[index + 2]
                
                if year_dic['year'] == str(year):
                    url_s = url['url']
                
        except:
            pass
        
    return url_s

In [7]:
# crawl rating data by beautiful soup
def get_content(url):
    html = get_html(url)
    soup = bs(html, 'lxml')
    data = {}
    try:
        try:
            data['critic(%)'] = soup.find('span', class_='superPageFontColor').span.text
        except:
            data['critic(%)'] = 'na'
            
        try:
            data['critic_rate'] = soup.find('div', id='scoreStats').findChildren()[0].text[25:].strip()
        except:
            data['critic_rate'] = 'na'
            
        try:
            data['critic_count'] = soup.find('div', id='scoreStats').findChildren()[2].text[18:].strip()
        except:
            data['critic_count'] = 'na'
            
        try:
            data['critic_fresh'] = soup.find('div', id='scoreStats').findChildren()[7].text
        except:
            data['critic_fresh'] = 'na'
        
        try:
            data['critic_rot'] = soup.find('div', id='scoreStats').findChildren()[10].text
        except:
            data['critic_rot'] = 'na'
            
        try:
            data['user(%)'] = soup.find('div', class_='audience-score').find('span', class_='superPageFontColor').text[:2]
        except:
            data['user(%)'] = 'na'
            
        try:
            data['user_rate'] = soup.find('div', class_='audience-info').findChildren()[0].contents[2].strip()
        except:
            data['user_rate'] = 'na'
        
        try:
            data['user_number'] = soup.find('div', class_='audience-info').findChildren()[2].contents[2].strip()
        except:
            data['user_number'] = 'na'

    except:
        print('error')
    
    return data

In [8]:
# get url function test
try_url = base_url + 'It'
data = get_movie_list(try_url)
data

'\n                        require([\'jquery\', \'globals\', \'search-results\', \'bootstrap\'], function($, RT, mount) {\n                            mount($("#search-results-root").get(0), RT.PrivateApiV2FrontendHost, \'It\', {"actorCount":225,"actors":[{"name":"Itzik Cohen","url":"/celebrity/itzik_cohen","image":"https://staticv2-4.rottentomatoes.com/static/images/redesign/poster_default_redesign.gif"},{"name":"Robert Ito","url":"/celebrity/robert_ito","image":"https://resizing.flixster.com/ZggedGKzSOKR1Lb59NkhfBvlKB8=/fit-in/80x80/v1.bjs4MDExODE7ajsxNzk2NDsxMjAwOzU0MDs3MjA"},{"name":"Miki Itô","url":"/celebrity/miki-ito","image":"https://staticv2-4.rottentomatoes.com/static/images/redesign/poster_default_redesign.gif"},{"name":"Italo Tajo","url":"/celebrity/italo_tajo","image":"https://staticv2-4.rottentomatoes.com/static/images/redesign/poster_default_redesign.gif"},{"name":"Faking It Three","url":"/celebrity/faking_it_three","image":"https://staticv2-4.rottentomatoes.com/static/i

In [ ]:
url_find = similar_check(data, 'It', '2017')
url_find